# 🚀 DOI to Reference Converter

Welcome to the web-based version of the DOI to Reference Converter! This tool allows you to easily convert a list of DOIs into a reference file (`.ris` or `.bib`) without any local installation.

## How to Use
1.  **Run the Setup Cell**: Execute the first code cell below (`Setup & Imports`). This will download the necessary code from the GitHub repository and prepare the environment. You only need to do this once per session.
2.  **Run the UI Cell**: Execute the second code cell (`User Interface`). This will display the interactive tool.
3.  **Generate Your File**: Use the tool to upload a `.txt` file or paste your DOIs, choose your format, and click the "Generate References" button. Your files will be downloaded automatically.


# Cell 1 : Setup & Imports

In [ ]:
# =================================================
# CELL 1: SETUP & IMPORTS
# =================================================
# Just run this cell once per session.

# 1. Clone the repository from GitHub
# IMPORTANT: REPLACE THE URL WITH YOUR OWN GITHUB REPO URL!
!git clone https://github.com/engkinandatama/doi-to-reference-converter.git

# 2. Change into the project directory
%cd doi-to-reference-converter

# 3. Install dependencies
!pip install -r requirements.txt

# 4. Import the core functions from the cloned converter.py file
from converter import clean_doi, get_metadata_from_doi
print("\n✅ Setup complete! You can now run the User Interface cell below." )

# Import other libraries needed for the UI
import requests
import re
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output
import time
from urllib.parse import quote
import io


# Cell 2: DOI_Converter_Functional

In [ ]:
# =================================================
# CELL 2: DOI_Converter_Functional
# =================================================
# Run this cell after the setup is complete to display the tool.

# --- Widget Definitions (Basic UI) ---
upload_button = widgets.FileUpload(
    accept='.txt', multiple=False, description='1. Upload .txt File', button_style='primary'
)
doi_input_area = widgets.Textarea(
    value='', placeholder='Or, paste your DOIs here, one per line...',
    description='2. Paste Manually:', layout={'height': '150px', 'width': '70%'},
    style={'description_width': 'initial'}
)
format_dropdown = widgets.Dropdown(
    options=[('RIS (for Mendeley/Zotero)', 'RIS'), ('BibTeX (for LaTeX)', 'BibTeX')],
    value='RIS', description='3. Select Format:', style={'description_width': 'initial'}
)
run_button = widgets.Button(
    description="🚀 Generate Files", button_style='success', tooltip='Start the conversion process'
)
output_area = widgets.Output()

# --- The function that runs when the button is clicked ---
def on_run_button_clicked(b):
    output_area.clear_output()
    with output_area: print("🚀 Starting process...\n")
    raw_dois = []
    if upload_button.value:
        uploaded_filename = list(upload_button.value.keys())[0]
        uploaded_file_content = upload_button.value[uploaded_filename]['content']
        raw_dois = io.TextIOWrapper(io.BytesIO(uploaded_file_content), encoding='utf-8').read().splitlines()
        with output_area: print(f"📂 Reading DOIs from file: {uploaded_filename}")
    elif doi_input_area.value:
        raw_dois = doi_input_area.value.split('\n')
        with output_area: print("📋 Reading DOIs from the text box.")
    if not raw_dois:
        with output_area: print("❗ Input is empty. Please upload a file or paste DOIs.")
        return
    cleaned_dois = sorted(list(set([clean_doi(d) for d in raw_dois if clean_doi(d)])))
    if not cleaned_dois:
        with output_area: print("❗ No valid DOIs found in your input.")
        return
    with output_area:
        print(f"🔍 Found {len(cleaned_dois)} unique DOIs to process.")
        print("-" * 40)
    successful_entries, failed_dois = [], []
    output_format = format_dropdown.value
    for doi in cleaned_dois:
        metadata = get_metadata_from_doi(doi, output_format) # Calling the imported function
        if metadata: successful_entries.append(metadata)
        else: failed_dois.append(doi)
        with output_area: print("-" * 40)

    # --- NEW & IMPROVED ENGLISH SUMMARY LOG ---
    with output_area:
        print("\n========================================")
        print("            PROCESS COMPLETE")
        print("========================================")
        print(f"Total DOIs Processed: {len(cleaned_dois)}")
        print(f"✅ Succeeded         : {len(successful_entries)}")
        print(f"❌ Failed            : {len(failed_dois)}")

        # If any DOIs failed, list them
        if failed_dois:
            print("\n----------------------------------------")
            print("List of DOIs that could not be found:")
            for i, doi in enumerate(failed_dois):
                print(f"  {i+1}. {doi}")
            print("----------------------------------------")
            print("\nA 'failed_dois.txt' file has also been created for your review.")
        else:
            print("\n✨ All DOIs were processed successfully! ✨")
        print("========================================")


    # --- File Download Section ---
    if successful_entries:
        file_extension = 'ris' if output_format == 'RIS' else 'bib'
        success_filename = f"successful_references.{file_extension}"
        with open(success_filename, "w", encoding="utf-8") as f: f.write("\n\n".join(successful_entries))
        files.download(success_filename)
        with output_area: print(f"\n✅ Downloading '{success_filename}'...")
    if failed_dois:
        fail_filename = "failed_dois.txt"
        with open(fail_filename, "w", encoding="utf-8") as f: f.write("\n".join(failed_dois))
        files.download(fail_filename)
        with output_area: print(f"🟡 Downloading '{fail_filename}' for your review.")

    # Reset the upload button
    if upload_button.value:
        upload_button.value.clear()
        upload_button._counter = 0

# --- Display all the widgets ---
run_button.on_click(on_run_button_clicked)
display(upload_button, doi_input_area, format_dropdown, run_button, output_area)

